#load and list the 29 zipcodes with top forcast returns of above 15% for years 2017-2021(based on prophet forcast) 
with open('pred_returns.pickle', 'rb') as f:
    top29zipcode_df = pickle.load(f)
    
zipcode_list = list(top29zipcode_df['RegionName'])

In [ ]:
#load the data train dataset
with open('train.pickle', 'rb') as f:
    train_df = pickle.load(f)

In [ ]:
#get the list of zipcodes present in both train dataset and zipcodes with top forcast returns
unique = list((train_df['RegionName'].unique()))
topzipcodes = list(set(zipcode_list).intersection(unique))
topzipcodes

In [ ]:
# import retrieving_zipcode_info. See as an example the dataframe for zipcode 32905
top29zipcode_dicts= fn.retrieving_zipcode_info(train_df, topzipcodes)

# Perform SARIMA

1. Perform a SARIMA model for an example zipcode 32905 and generate output model plots. 
2. Perform a SARIMA model for all zipcodes.

## Perform an initial SARIMA model for an example zipcode 32905 and generate output model plots: 

In [ ]:
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q.
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q.
pdqs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [ ]:
# Run a grid with pdq and seasonal pdq parameters calculated above and get the best AIC value for an example zipcode 94085. 
ans = []
for comb in pdq:
    for combs in pdqs:
        try:
            mod = sm.tsa.statespace.SARIMAX(top29zipcode_dicts[94085],
                                            order=comb,
                                            seasonal_order=combs,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            output = mod.fit()
            ans.append([comb, combs, output.aic])
            print('ARIMA {} x {}12 : AIC Calculated ={}'.format(comb, combs, output.aic))
        except:
            continue
            
# Find the parameters with minimal AIC value.
ans_df = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'aic'])
ans_df.loc[ans_df['aic'].idxmin()]


#perform a SARIMA model on example zipcode 94085

SARIMA_MODEL = sm.tsa.statespace.SARIMAX(top29zipcode_dicts[94085],
                                order=(1, 1, 1),
                                seasonal_order=(1, 1, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

output = SARIMA_MODEL.fit()

print(output.summary().tables[1])

In [ ]:
#plot output to evaluate 
output.plot_diagnostics(figsize=(15, 18))
plt.show()

In [ ]:
# Get forecast for 60 steps ahead in future for zipcode 94085
prediction = output.get_forecast(steps=60)

# Get confidence intervals of forecasts for zipcode 94085
pred_conf = prediction.conf_int()
pred_conf.tail()

In [ ]:
#The plot suggests the predicted mean of the median prices and 95% CIs for the example zipcode 94085. 

ax = top29zipcode_dicts[94085].plot(label='observed', figsize=(20, 15))
prediction.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_conf.index,
                pred_conf.iloc[:, 0],
                pred_conf.iloc[:, 1], color='k', alpha=.25)
ax.set_xlabel('Date')
ax.set_ylabel('Price')

plt.legend(['observed', 'forcast'])
plt.show()

## Perform a SARIMA model for all zipcodes.

In [ ]:
pred_29zipcodes={}
for zipcode in top29zipcode_dicts.keys():
    pred_29zipcodes[zipcode]= fn.model_SARIMA_zipcode(top29zipcode_dicts[zipcode],pdq,pdqs)

In [ ]:
merged = pd.DataFrame(data=None)
for i in pred_29zipcodes.keys():
    df=pd.DataFrame()
    df['date']= pred_29zipcodes[i].conf_int().index
    df['RegionName'] = i
    df['pred_mean'] = np.array(pred_29zipcodes[i].predicted_mean)
    df['lower_value']=np.array(pred_29zipcodes[i].conf_int()['lower value'])
    df['upper_value']=np.array(pred_29zipcodes[i].conf_int()['upper value'])
    merged = pd.concat([merged, df], axis=0)

In [ ]:
merged.tail()

## Calculate annualized returns for SARIMA model forcasts from 2018 to 2023 for all 20 zipcodes.

I calculated the annualized returns for SARIMA model forcasts between years 2018 to 2023 for all 29 zipcodes.

In [ ]:
#subset the merge dataframe to only get the data from 2018-07-01 to 2022-07-01
years = range(2018, 2023)
year_month_list = [datetime.strptime(f'{year}-07-01', '%Y-%m-%d').date() for year in years]
forecast_returns = merged.loc[merged['date'].isin(year_month_list)]
forecast_returns.head()

In [ ]:
#subset the merge dataframe to only get the data from 2018 to 2023.
forecast_returns['returns'] = forecast_returns['pred_mean'].div(
    forecast_returns.groupby('RegionName')['pred_mean'].shift(1))

In [ ]:
forecast_returns = forecast_returns.dropna(subset=['returns'])

Run the function annualised_returns below to get a dataframe consisting of predicted annualized returns for all zipcodes (29).  

In [ ]:
forecast_returns = fn.annualised_returns(forecast_returns)

In [ ]:
print(f"Sorted predicted annual returns: {forecast_returns['Ann_returns']}")

In [ ]:
with open('forecast_returns.pickle', 'wb') as f:
    pickle.dump(forecast_returns, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('pred_29zipcodes.pickle', 'wb') as f:
    pickle.dump(pred_29zipcodes, f, pickle.HIGHEST_PROTOCOL)